In [ ]:
!pip install pyngrok
!apt-get install -y xvfb x11-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.1-py3-none-any.whl size=21122 sha256=5593f24e7843b2bb89fe07e068dbdbc36c8b4da7ab6f0b4b666995d26901229f
  Stored in directory: /root/.cache/pip/wheels/3b/32/0e/27789b6fde02bf2b320d6f1a0fd9e1354b257c5f75eefc29bc
Successfully built pyngrok
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 libxtst6 libxxf86dga1 x11-utils x11-xkb-utils xfonts-base
  xfonts-encodings xfonts-utils xserver-common xvfb
0 upgraded, 12 newly installed, 0 to remove and 15 not upgraded.
Need to get 8,045 kB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Lambda, Embedding, Dense, GlobalAveragePooling1D, Input
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
import os


In [ ]:
import numpy as np
import pickle

# Load padded_sequences and embedding_matrix from disk
padded_X = np.load('/content/drive/MyDrive/padded_X.npy')
with open('/content/drive/MyDrive/embedding_matrix.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

In [ ]:
import pandas as pd

file_path = "/content/drive/MyDrive/y.csv"

# Load data into a DataFrame
y = pd.read_csv(file_path)

# Display the DataFrame
print(y)

                Case_Type_str
0          Administrative Law
1          Administrative Law
2          Administrative Law
3          Administrative Law
4          Administrative Law
...                       ...
39150   Workers' Compensation
39151   Workers' Compensation
39152   Workers' Compensation
39153   Workers' Compensation
39154   Workers' Compensation

[39155 rows x 1 columns]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(padded_X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

# Assuming y_train is your categorical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.fit_transform(y_val)
y_test_encoded = label_encoder.fit_transform(y_test)

# Reshape y_train_encoded to a column vector
y_train_encoded_reshaped = np.array(y_train_encoded).reshape(-1, 1)
y_val_encoded_reshaped = np.array(y_val_encoded).reshape(-1, 1)
y_test_encoded_reshaped = np.array(y_test_encoded).reshape(-1, 1)

# Use OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
y_train_onehot = onehot_encoder.fit_transform(y_train_encoded_reshaped)
y_val_onehot = onehot_encoder.fit_transform(y_val_encoded_reshaped)
y_test_onehot = onehot_encoder.fit_transform(y_test_encoded_reshaped)
y_train_onehot

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignor

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
X_train.shape

(27408, 200)

In [ ]:
y_train_onehot.shape

(27408, 76)

In [ ]:
vocabulary_size, embedding_dimension = embedding_matrix.shape

In [ ]:
print(f'Vocabulary size: {vocabulary_size}')
print(f'Embedding dimension: {embedding_dimension}')

Vocabulary size: 268458
Embedding dimension: 200


In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
num_classes = 76

# Create the model
model = Sequential()
model.add(Embedding(input_dim=vocabulary_size, output_dim=embedding_dimension, input_length=200))
model.add(LSTM(150))  # Adjust the number of LSTM units as needed
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_onehot, epochs=20, batch_size=32, validation_data=(X_val, y_val_onehot), callbacks=[early_stopping])


Epoch 1/20
857/857 [==============================] - 58s 59ms/step - loss: 3.8860 - accuracy: 0.0903 - val_loss: 3.5531 - val_accuracy: 0.1514
Epoch 2/20
857/857 [==============================] - 20s 24ms/step - loss: 3.3988 - accuracy: 0.1927 - val_loss: 3.2544 - val_accuracy: 0.2231
Epoch 3/20
857/857 [==============================] - 15s 17ms/step - loss: 2.8553 - accuracy: 0.3024 - val_loss: 3.4070 - val_accuracy: 0.2079
Epoch 4/20
857/857 [==============================] - 14s 16ms/step - loss: 2.4449 - accuracy: 0.3912 - val_loss: 2.8712 - val_accuracy: 0.3181
Epoch 5/20
857/857 [==============================] - 14s 17ms/step - loss: 2.0210 - accuracy: 0.4879 - val_loss: 2.9059 - val_accuracy: 0.3233
Epoch 6/20
857/857 [==============================] - 14s 16ms/step - loss: 1.6004 - accuracy: 0.5870 - val_loss: 2.9997 - val_accuracy: 0.3395
Epoch 7/20
857/857 [==============================] - 13s 15ms/step - loss: 1.1923 - accuracy: 0.6926 - val_loss: 3.1705 - val_accuracy:

In [ ]:
from sklearn.metrics import accuracy_score

y_test_decoded = np.argmax(y_test_onehot, axis=1)

# Get model predictions on the test set
y_pred = model.predict(X_test)
y_pred_decoded = np.argmax(y_pred, axis=1)

# Calculate accuracy
test_accuracy = accuracy_score(y_test_decoded, y_pred_decoded)

# Print the test accuracy
print(f'Test Accuracy: {test_accuracy:.4f}')

184/184 [==============================] - 1s 4ms/step
Test Accuracy: 0.3459


In [ ]:
model.save_weights('/content/drive/MyDrive/model1_weights.h5')

In [25]:

file_path = "/content/drive/MyDrive/data.csv"

# Load data into a DataFrame
data = pd.read_csv(file_path)
data = data.drop(columns=['Unnamed: 0', 'Case_description'], axis=1)
data['Case_Type'] = data['Case_Type'].astype(int)

# Display the DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39155 entries, 0 to 39154
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Case_Type              39155 non-null  int64 
 1   processed_description  39154 non-null  object
dtypes: int64(1), object(1)
memory usage: 611.9+ KB


In [30]:
# Create an embedding layer
embedding_layer = Embedding(
    input_dim = vocabulary_size,
    output_dim = embedding_dimension,
    embeddings_initializer=Constant(embedding_matrix),
    input_length= embedding_dimension,
    trainable=True  # set to True if you want to fine-tune embeddings during training
)

In [31]:
import tensorflow as tf
from keras.layers import LayerNormalization, MultiHeadAttention, Dense, LSTM, Bidirectional

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rnn_units, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

        # RNN as Encoder
        self.rnn_encoder = Bidirectional(LSTM(units=rnn_units, return_sequences=True))

        # LSTM as Decoder
        self.lstm_decoder = LSTM(units=rnn_units, return_sequences=True)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [32]:
# Create the model using TransformerBlock
num_heads = 2
ff_dim = 32
rnn_units = 64

transformer_block = TransformerBlock(embedding_dimension, num_heads, ff_dim, rnn_units)

In [36]:
# Define the model
inputs = Input(shape=(200,))
embedded_inputs = Embedding(
    input_dim=vocabulary_size,
    output_dim=embedding_matrix.shape[1],
    embeddings_initializer=Constant(embedding_matrix),
    input_length=200,
    trainable=True
)(inputs)

#inputs = Input(shape=vocabulary_size)
embedding_output = embedding_layer(inputs)
transformer_block = TransformerBlock(embedding_matrix.shape[1], num_heads=2, ff_dim=32, rnn_units=64)
encoder_outputs = transformer_block(embedding_output)
outputs = Dense(200, activation='softmax')(encoder_outputs)


In [39]:
# Define the model
sentence_model = Model(inputs=inputs, outputs=outputs)
sentence_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 200)          53691600  
                                                                 
 transformer_block_3 (Trans  (None, 200, 200)          335232    
 formerBlock)                                                    
                                                                 
 dense_13 (Dense)            (None, 200, 200)          40200     
                                                                 
Total params: 54067032 (206.25 MB)
Trainable params: 54067032 (206.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Compile the model
sentence_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
sentence_model.fit(X_train, X_train, epochs=20, batch_size=12, validation_data=(X_val, X_val))

Epoch 1/20
2284/2284 [==============================] - 46s 19ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/20
2284/2284 [==============================] - 22s 10ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/20
2284/2284 [==============================] - 22s 10ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/20
2284/2284 [==============================] - 21s 9ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/20
2284/2284 [==============================] - 21s 9ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/20
2284/2284 [==============================] - 21s 9ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/20
2284/2284 [==============================] - 21s 9ms/step - loss: nan - accuracy: 3.6486e-05 - val_loss:

In [43]:
sentence_model.save_weights('/content/drive/MyDrive/model2_weights.h5')

In [44]:
sentence_model.save("/content/drive/MyDrive/model2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
